In [ ]:
!pip install pandas==1.5.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

print(pd.__version__)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_pickle('/content/drive/MyDrive/train_features_legalBert_preamble.pkl')
df2 = pd.read_pickle('/content/drive/MyDrive/dev_features_legalBert_preamble.pkl')

In [ ]:
df2.head()

In [ ]:
features = df[['features']].to_numpy()
labels = df[['labels']].to_numpy()

all_features = []
all_labels = []
for i in range(len(df)):
  all_features.append(np.array(features[i][0]))
  all_labels.append(np.array(labels[i][0]))

all_features = np.array(all_features)
all_labels = np.array(all_labels)

In [ ]:
dev_feat = df[['features']].to_numpy()
dev_lab = df[['labels']].to_numpy()

feature_dev = []
label_dev = []
for i in range(len(df2)):
  feature_dev.append(np.array(dev_feat[i][0]))
  label_dev.append(np.array(dev_lab[i][0]))

feature_dev = np.array(feature_dev)
label_dev = np.array(label_dev)

In [ ]:
print(len(feature_dev), len(label_dev))

In [ ]:
print(all_features.shape)

In [ ]:
from keras.utils import to_categorical
import numpy as np


def reshape_features_and_convert_labels(features, labels, num_classes):
  feat = np.reshape(
    features, (features.shape[0], features.shape[2], features.shape[3]))
  
  lab = to_categorical(labels, num_classes=num_classes)
  return feat, lab


train_features, train_labels = reshape_features_and_convert_labels(all_features, all_labels, 17)
test_features, test_labels = reshape_features_and_convert_labels(feature_dev, label_dev, 17)

half_train1_features = train_features[:len(train_features)//2]
half_train1_labels = train_labels[:len(train_labels)//2]

half_train2_features = train_features[len(train_features)//2:]
half_train2_labels = train_labels[len(train_labels)//2:]

print(half_train1_features.shape)
print(half_train1_labels.shape)

print(train_features.shape)
print(train_labels.shape)

print(test_features.shape)
print(test_labels.shape)

# print(train_labels)

In [ ]:
from keras.backend import dropout
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Activation, Dropout
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(512, 512)))
model.add(Dropout(0.5))
model.add(LSTM(8, return_sequences=True))
model.add(Dense(17))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy', 
                                         tf.keras.metrics.Precision(), 
                                         tf.keras.metrics.Recall()])
model.summary()

In [ ]:
history1 = model.fit(x=half_train1_features, 
                     y=half_train1_labels, batch_size=250, 
                     epochs=20)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history1.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train1', 'train2'])
plt.show()
# summarize history for loss
plt.plot(history1.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train1', 'train2'])
plt.show()

In [ ]:
# half_test_features = train_features[len(train_features)//2:900]
# half_test_labels = train_labels[len(train_labels)//2:900]

# print(half_test_features.shape)
# print(half_test_labels.shape)


In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_features, test_labels)
print("test loss, test acc:", results)

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


predictions = model.predict(test_features)
predicted_labels = np.argmax(predictions, axis=2)
# print(predicted_labels)
print(len(predicted_labels))
print(predicted_labels[1])

print(classification_report(test_features, predictions))

# Tlabels = np.argmax(half_test_labels, axis=2)
# y_predict = np.argmax(model.predict(half_test_features), axis=2)
# print(classification_report(Tlabels, y_predict))



# label_dict = {
#     'accident_Cropped': 0,
#     'call_Cropped': 1,
#     'doctor_Cropped': 2,
#     'help_Cropped': 3,
#     'hot_Cropped': 4,
#     'lose_Cropped': 5,
#     'pain_Cropped': 6,
#     'thief_Cropped': 7,
# }

# # print([str(label_dict[i])+":"+str(label_dict[j]) for i,j in enumerate(Tlabels, predicted_labels)])

# matrix_confusion = confusion_matrix(Tlabels, predicted_labels)
# matrix_confusion

# sns.heatmap(matrix_confusion.T, square = True, annot = True, fmt = 'd', cbar = False)
# plt.xlabel("True label") 
# plt.ylabel("Predicted label")

# print(classification_report(Tlabels, predicted_labels, target_names=['accident', 'call', 'doctor', 'help', 'hot', 'lose', 'pain', 'thief']))

In [ ]:
# print(half_test_features[0])
# print(y_predict[0])

In [ ]:
import pandas as pd

final_df = pd.read_csv('/content/drive/MyDrive/Final_DataFrame_Dev_Preamble')



In [ ]:
train_sents = final_df['clean_sent'].to_list()
train_labels = final_df['tags'].to_list()

print(len(train_sents))
print(len(train_labels))

train_sents = [train_sents[i].replace("'", "").strip(
    '][').split(', ') for i in range(len(train_sents))]
train_labels = [train_labels[i].replace("'", "").strip(
    '][').split(', ') for i in range(len(train_labels))]


print(len(train_sents[0]))
print(train_labels[8])




In [ ]:
print([sent for sent in train_sents])

In [ ]:

# def get_dict_map(df , token_or_tag):
#     """
#     converts word to index

#     [Need changes !!!]
#     """
#     tok2idx = {}
#     idx2tok = {}

#     vocab = []

#     if token_or_tag == 'token':
#         for sent in df:
#           vocab.extend( list(set(sent)) )

#         vocab = set(vocab)

#     else :
#         for sent in df:
#           vocab.extend( list(set(sent)) )
        
#         vocab = set(vocab)


#     idx2tok = {idx:tok for idx , tok in enumerate(vocab)}
#     tok2idx = {tok:idx for idx , tok in enumerate(vocab)}

#     return tok2idx, idx2tok




In [ ]:
import json

f = open('/content/drive/MyDrive/tag2idx_Preamble')
tag2idx = json.load(f)
# Closing file
f.close()

In [ ]:
print(tag2idx)

In [ ]:
import json

f = open('/content/drive/MyDrive/idx2tag_Preamble')
idx2tag = json.load(f)
# Closing file
f.close()


In [ ]:
print(idx2tag)

In [ ]:
# word2idx, idx2word = get_dict_map(train_sents, 'token')
# tag2idx, idx2tag = get_dict_map(train_labels, 'tag')

In [ ]:
print(idx2tag)
print(idx2tag[str(5)])

In [ ]:
def conversion(prediction, idx2tag):
  tag_seq = []
  for i,p in enumerate(prediction):
    if p != 16:
      tag_seq.append(idx2tag[str(p)])
  return tag_seq

In [ ]:
tags = conversion(predicted_labels[18], idx2tag)
o_tags = conversion(label_dev[18], idx2tag)

for tag in tags:
  print(tag, end=",")

In [ ]:
print(tags)
print(o_tags)